In [5]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

In [6]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate() + LoggerMessageGate())
    
    x_prev = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1  = add!(model, constvar(:add_1, 1.0))
    x      = add!(model, randomvar(:x))
    
    x_prev_add = add!(model, make_node(+, x_prev, add_1, x))
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    y     = add!(model, datavar(:y, Float64))
    
    add_x_and_noise = add!(model, make_node(+, x, noise, y))
    
    activate!(model)
    
    return x_prev, x, y
end

function kalman(data)
    N = length(data)
    
    x_prev, x, y = kalman_filter_graph()

    link(x_prev, getmarginal(x))
    
    update!(x_prev, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [7]:
N = 10
data = collect(1:N) + sqrt(200.0) * randn(N);

In [8]:
kalman(data)

From variable out of node + => Normal{Float64}(μ=1.0, σ=100.0)
From variable in1 of node + => Normal{Float64}(μ=7.835323108221076, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=8.701297164922625, σ=14.002800840280099)
From variable in1 of node + => Normal{Float64}(μ=-7.035624128174767, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=1.9107420693298571, σ=9.950371902099892)
From variable in1 of node + => Normal{Float64}(μ=-1.7860968664321764, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=1.6866232164285213, σ=8.137884587711596)
From variable in1 of node + => Normal{Float64}(μ=17.615051072082636, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=6.648918702909646, σ=7.053456158585984)
From variable in1 of node + => Normal{Float64}(μ=-4.287128816511396, σ=14.142135623730951)
From variable out of node + => Normal{Float64}(μ=5.470423181112626, σ=6.311944030978033)
From variable in1 of node + => Nor

10-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=7.701297164922624, σ=14.002800840280099)
 Normal{Float64}(μ=0.9107420693298571, σ=9.950371902099892)
 Normal{Float64}(μ=0.6866232164285214, σ=8.137884587711596)
 Normal{Float64}(μ=5.648918702909646, σ=7.053456158585984)
 Normal{Float64}(μ=4.470423181112626, σ=6.311944030978033)
 Normal{Float64}(μ=4.757265988952403, σ=5.763904177042351)
 Normal{Float64}(μ=3.9650519375514515, σ=5.337605126836238)
 Normal{Float64}(μ=7.4650660698607405, σ=4.993761694389224)
 Normal{Float64}(μ=6.427513928089824, σ=4.708816093480111)
 Normal{Float64}(μ=8.646449807111365, σ=4.467670516087703)